In [1]:
from datasets import load_dataset, load_metric
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML
import re
import json
from transformers import Wav2Vec2CTCTokenizer
from transformers import Wav2Vec2FeatureExtractor
from transformers import Wav2Vec2Processor
import IPython.display as ipd
import numpy as np
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import Wav2Vec2ForCTC
from transformers import TrainingArguments
from transformers import Trainer

Dataset

In [2]:
libri= load_dataset("librispeech_asr", name="clean")
print(libri)

Reusing dataset librispeech_asr (/home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/14c8bffddb861b4b3a4fcdff648a56980dbb808f3fc56f5a3d56b18ee88458eb)


  0%|          | 0/4 [00:00<?, ?it/s]

DatasetDict({
    train.100: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 28539
    })
    train.360: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 104014
    })
    validation: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 2703
    })
    test: Dataset({
        features: ['file', 'audio', 'text', 'speaker_id', 'chapter_id', 'id'],
        num_rows: 2620
    })
})


In [3]:
libri = libri.remove_columns(['speaker_id', 'chapter_id', 'id'])

In [4]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

show_random_elements(libri["train.100"].remove_columns(["file", "audio"]))

,text
0,SHE SEIZED WITH AN UNSTEADY HAND THE PRECIOUS MANUSCRIPT
1,AND APPARENTLY SUPPORTED THE REBELLIONS OF THE NORTHERN MESOPOTAMIAN KINGS
2,SAMSON'S INTENTION IN PERSUADING HIM TO SALLY FORTH ONCE MORE WAS TO DO WHAT THE HISTORY RELATES FARTHER ON
3,AND AS I HAD ALREADY PREPARED EVERYTHING AS WELL AS I COULD AS TO MY BUSINESS AND WHOM TO ENTRUST MY AFFAIRS WITH I HAD LITTLE TO DO BUT TO RESOLVE I WENT HOME THAT EVENING GREATLY OPPRESSED IN MY MIND
4,NO OTHER MAN IN ALL THIS FRANCE COULD HAVE DONE WHAT HE HAS DONE FOR ME SHE LAID HER HEAD UPON HER FATHER'S BREAST AS SHE HAD LAID HIS POOR HEAD ON HER OWN BREAST LONG LONG AGO HE WAS HAPPY IN THE RETURN HE HAD MADE HER
5,TO THE SUPPORT OF ITS ORDINANCES AND NOT TO BE COMPELLED AFTER SUCH A FASHION THERE I HAVE YOU EXCLAIMED MISTER TEMPLETON
6,SIX MEN WHO WERE MARCHING SEPARATELY AND AT SOME DISTANCE FROM EACH OTHER ALONG THE WALL AND WHO MIGHT HAVE BEEN TAKEN FOR A GRAY PATROL
7,SUPPER WAS NEARLY READY WHEN MISTER PARKER RETURNED HIS FACE WORE A RATHER SERIOUS AIR AND MISTER DAMON NOTING IT ASKED LAUGHINGLY WELL
8,WE MAY REJOICE THAT THESE CAUSES OF IRRITATION NO LONGER EXIST AND THAT IF SUCH FEELINGS AS JEALOUSY RIVALRY AND DISCONTENT EVER TOUCH CELESTIAL BOSOMS IN THE MODERN BALL ROOM THEY MUST ARISE FROM DIFFERENT AND MORE RECONDITE SOURCES
9,IN THE DIVISION OF THE BAUBLES THE CUNNING HURON DISCOVERED NO LESS ART THAN IN THEIR SELECTION WHILE HE BESTOWED THOSE OF GREATER VALUE ON THE TWO MOST DISTINGUISHED WARRIORS ONE OF WHOM WAS HIS HOST HE SEASONED HIS OFFERINGS TO THEIR INFERIORS


In [5]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

libri = libri.map(remove_special_characters)

Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/14c8bffddb861b4b3a4fcdff648a56980dbb808f3fc56f5a3d56b18ee88458eb/cache-f2f76a6a936a78ee.arrow
Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/14c8bffddb861b4b3a4fcdff648a56980dbb808f3fc56f5a3d56b18ee88458eb/cache-cc8bea45c46a7c7c.arrow
Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/14c8bffddb861b4b3a4fcdff648a56980dbb808f3fc56f5a3d56b18ee88458eb/cache-887227bc6ed99a36.arrow
Loading cached processed dataset at /home/ujan/.cache/huggingface/datasets/librispeech_asr/clean/2.1.0/14c8bffddb861b4b3a4fcdff648a56980dbb808f3fc56f5a3d56b18ee88458eb/cache-090e62f70d2d3247.arrow


In [6]:
show_random_elements(libri["train.100"].remove_columns(["file", "audio"]))

,text
0,regardless of its effect on the mate is short sighted in the extreme even if you could care only for personal pleasure that cannot long outlast your spouse's displeasure staging a contest or a succession of small contests for the sake of finding out who is boss
1,he's such a game little feller an' an such a queer little un and when they turned away he stood and looked after them in a dazed kind of way and there was still a mist in his eyes and a lump in his throat
2,anyone could have told her that by this attitude of mind towards the future she was laying up for herself disappointment at the least if not the bitterest disillusions but there was no one to throw cold water on her hopes
3,he's axtra fond o the feeshin was the reply for a that he's a foreign shentleman waiting till the boat had become a distant speck on the face of the waters gimblet made his way into the inn and entered into conversation with the landlord
4,and of the hethites and of the hevites and of the amorrhites and all the mighty ones in hesebon
5,the life is not so bad said craggs but it's lonely sometimes life need never be lonely so long as a man has health and his faculties said billy give me nature to admire a bit of baycon for dinner
6,as a continuous accompaniment to the exclamations sighs and whispered comments of the bystanders came the murmur of the water trickling from a spigot at the head of each slab a tiny stream that flowed forth
7,you know nothing of her former life and last year at this time you did not know that such a person existed no matter there is such a thing as looking through a person's eyes into the heart and learning more of the height and breadth
8,exposure threatening you at any moment why don't you tell me again that i lie carling but now the man made no answer he had sunk a little deeper in his chair
9,riquet with the tuft cinderella and little thumb eight stories in all on the cover of the book was depicted an old lady holding in her hand a distaff


Extract all distinct letters of the training and test data and build our vocabulary from this set of letters

In [7]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

vocabs = libri.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=libri.column_names["train.100"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
vocab_list = list(set(vocabs["train.100"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'p': 0,
 'r': 1,
 'g': 2,
 'z': 3,
 'o': 4,
 'u': 5,
 'd': 6,
 'b': 7,
 'k': 8,
 'e': 9,
 'y': 10,
 's': 11,
 'c': 12,
 'w': 13,
 'h': 14,
 'a': 15,
 't': 16,
 'x': 17,
 ' ': 18,
 'v': 19,
 'i': 20,
 'm': 21,
 "'": 22,
 'l': 23,
 'n': 24,
 'j': 25,
 'f': 26,
 'q': 27}

To make it clearer that " " has its own token class, we give it a more visible character |. In addition, we also add an "unknown" token. We also add a padding token that corresponds to CTC's "blank token"

In [9]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


Save the vocabulary as a json file

In [10]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

We use the json file to instantiate an object of the Wav2Vec2CTCTokenizer class.

In [11]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

Wav2Vec2 feature extractor

In [12]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

To make the usage of Wav2Vec2 as user-friendly as possible, the feature extractor and tokenizer are wrapped into a single Wav2Vec2Processor class so that one only needs a model and processor object

In [13]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [14]:
libri['train.100'][0]

{'file': '/home/ujan/.cache/huggingface/datasets/downloads/extracted/02733bd6de54bdc39dd89183579973aa11646dce57a22216427906405ebc7284/374-180298-0000.flac',
 'audio': {'path': '/home/ujan/.cache/huggingface/datasets/downloads/extracted/02733bd6de54bdc39dd89183579973aa11646dce57a22216427906405ebc7284/374-180298-0000.flac',
  'array': array([ 7.01904297e-04,  7.32421875e-04,  7.32421875e-04, ...,
         -2.74658203e-04, -1.83105469e-04, -3.05175781e-05]),
  'sampling_rate': 16000},
 'text': 'chapter sixteen i might have told you of the beginning of this liaison in a few lines but i wanted you to see every step by which we came i to agree to whatever marguerite wished'}

In [15]:
rand_int = random.randint(0, len(libri["train.100"]))

print(libri["train.100"][rand_int]["text"])
ipd.Audio(data=np.asarray(libri["train.100"][rand_int]["audio"]["array"]), autoplay=True, rate=16000)

there was a moon that night wasn't there full or near it i returned and how soon was the alarm raised along the countryside


In [16]:
rand_int = random.randint(0, len(libri["train.100"]))

print("Target text:", libri["train.100"][rand_int]["text"])
print("Input array shape:", np.asarray(libri["train.100"][rand_int]["audio"]["array"]).shape)
print("Sampling rate:", libri["train.100"][rand_int]["audio"]["sampling_rate"])

Target text: take a golden walnut and hide it in my green box ask the young lady olga ignatyevna for it say it's for vanka vanka sighed convulsively and again stared at the window
Input array shape: (225600,)
Sampling rate: 16000


In [17]:
def prepare_dataset(batch):
    audio = batch["audio"]

    # batched output is "un-batched" to ensure mapping is correct
    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    
    with processor.as_target_processor():
        batch["labels"] = processor(batch["text"]).input_ids
    return batch

In [18]:
libri = libri.map(prepare_dataset, remove_columns=libri.column_names["train.100"], num_proc=4)

#0:   0%|          | 0/7135 [00:00<?, ?ex/s]

#1:   0%|          | 0/7135 [00:00<?, ?ex/s]

#2:   0%|          | 0/7135 [00:00<?, ?ex/s]

#3:   0%|          | 0/7134 [00:00<?, ?ex/s]

#0:   0%|          | 0/26004 [00:00<?, ?ex/s]

...

#2:   0%|          | 0/676 [00:00<?, ?ex/s]

#0:   0%|          | 0/655 [00:00<?, ?ex/s]

#1:   0%|          | 0/655 [00:00<?, ?ex/s]

#2:   0%|          | 0/655 [00:00<?, ?ex/s]

#3:   0%|          | 0/655 [00:00<?, ?ex/s]

Training & Evaluation

In [19]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [20]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [21]:
wer_metric = load_metric("wer")

In [22]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

In [23]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/configuration_utils.py:368: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['project_q.bias', 'project_hid.weight', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'quantizer.codevectors', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a

The first component of Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretrainind and as stated in the paper does not need to be fine-tuned anymore 

In [24]:
model.freeze_feature_extractor()

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:1618: FutureWarning: The method `freeze_feature_extractor` is deprecated and will be removed in Transformers v5.Please use the equivalent `freeze_feature_encoder` method instead.
  warnings.warn(


In [25]:
training_args = TrainingArguments(
  output_dir='/home/ujan/Notebooks/libri_fn',
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  gradient_checkpointing=True, 
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [26]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=libri["train.100"],
    eval_dataset=libri["test"],
    tokenizer=processor.feature_extractor,
)

Using cuda_amp half precision backend


In [27]:
trainer.train()

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 28539
  Num Epochs = 30
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 26760


Step,Training Loss,Validation Loss,Wer
500,3.696200,2.000828,0.950339
1000,0.399600,0.332512,0.225008
1500,0.153500,0.171447,0.155394
2000,0.117600,0.467462,0.173444
2500,0.096600,0.146005,0.131714
3000,0.081600,0.164864,0.138980
3500,0.077800,0.140615,0.128081
4000,0.068300,0.143121,0.125818
4500,0.064200,0.137573,0.119028
5000,0.058500,0.173645,0.139550


***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/libri_fn/checkpoint-500
Configuration saved in /home/ujan/Notebooks/libri_fn/checkpoint-500/config.json
Model weights saved in /home/ujan/Notebooks/libri_fn/checkpoint-500/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/libri_fn/checkpoint-500/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/libri_fn/checkpoint-1000
Configuration saved in /home/ujan/Notebooks/libri_fn/checkpoint-1000/config.json
Model weights saved in /home/ujan/Notebooks/libri_fn/checkpoint-1000/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/libri_fn/checkpoint-1000/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/libri_fn/checkpoint-1500
Configuration saved in /home/ujan/Notebooks

Model weights saved in /home/ujan/Notebooks/libri_fn/checkpoint-8500/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/libri_fn/checkpoint-8500/preprocessor_config.json
Deleting older checkpoint [/home/ujan/Notebooks/libri_fn/checkpoint-7500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/libri_fn/checkpoint-9000
Configuration saved in /home/ujan/Notebooks/libri_fn/checkpoint-9000/config.json
Model weights saved in /home/ujan/Notebooks/libri_fn/checkpoint-9000/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/libri_fn/checkpoint-9000/preprocessor_config.json
Deleting older checkpoint [/home/ujan/Notebooks/libri_fn/checkpoint-8000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/libri_fn/checkpoint-9500
Configuration saved in /home/ujan/Notebooks/libri_fn/chec

Model weights saved in /home/ujan/Notebooks/libri_fn/checkpoint-16500/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/libri_fn/checkpoint-16500/preprocessor_config.json
Deleting older checkpoint [/home/ujan/Notebooks/libri_fn/checkpoint-15500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/libri_fn/checkpoint-17000
Configuration saved in /home/ujan/Notebooks/libri_fn/checkpoint-17000/config.json
Model weights saved in /home/ujan/Notebooks/libri_fn/checkpoint-17000/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/libri_fn/checkpoint-17000/preprocessor_config.json
Deleting older checkpoint [/home/ujan/Notebooks/libri_fn/checkpoint-16000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/libri_fn/checkpoint-17500
Configuration saved in /home/ujan/Notebooks/libr

Model weights saved in /home/ujan/Notebooks/libri_fn/checkpoint-24500/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/libri_fn/checkpoint-24500/preprocessor_config.json
Deleting older checkpoint [/home/ujan/Notebooks/libri_fn/checkpoint-23500] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/libri_fn/checkpoint-25000
Configuration saved in /home/ujan/Notebooks/libri_fn/checkpoint-25000/config.json
Model weights saved in /home/ujan/Notebooks/libri_fn/checkpoint-25000/pytorch_model.bin
Feature extractor saved in /home/ujan/Notebooks/libri_fn/checkpoint-25000/preprocessor_config.json
Deleting older checkpoint [/home/ujan/Notebooks/libri_fn/checkpoint-24000] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 2620
  Batch size = 8
Saving model checkpoint to /home/ujan/Notebooks/libri_fn/checkpoint-25500
Configuration saved in /home/ujan/Notebooks/libr

TrainOutput(global_step=26760, training_loss=0.11166186776396406, metrics={'train_runtime': 90805.915, 'train_samples_per_second': 9.429, 'train_steps_per_second': 0.295, 'total_flos': 9.971972038207242e+19, 'train_loss': 0.11166186776396406, 'epoch': 30.0})

Evaluation

In [28]:
def map_to_result(batch):
  with torch.no_grad():
    input_values = torch.tensor(batch["input_values"], device="cuda").unsqueeze(0)
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]
  batch["text"] = processor.decode(batch["labels"], group_tokens=False)
  
  return batch

results = libri["test"].map(map_to_result, remove_columns=libri["test"].column_names)

  0%|          | 0/2620 [00:00<?, ?ex/s]

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [29]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["text"])))

Test WER: 0.082


In [32]:
show_random_elements(results)

,pred_str,text
0,my heart doth plead that thou in him dost lie a closet never pierced with crystal eyes but the defendant doth that plead deniye and says in him thy fair apparants lies,my heart doth plead that thou in him dost lie a closet never pierc'd with crystal eyes but the defendant doth that plea deny and says in him thy fair appearance lies
1,she was here just now said the count,she was here just now said the count
2,but hackey who was too much occupied with his own thoughts to note the movement continued speaking more to himself than to his companion,but hawkeye who was too much occupied with his own thoughts to note the movement continued speaking more to himself than to his companion
3,a word sha now be said about the origin of luther's commonto agelations,a word should now be said about the origin of luther's commentary on galatians
4,you have received us with all that courtesy and hospitality for which your character in england stands so high,you have received us with all that courtesy and hospitality for which your character in england stands so high
5,so we harried the cost of norway,so we harried the coast of norway
6,for many then this book has been a source of fascination surely one of the most influential novels ever written an inspiration for such scientists and discoverers as engineer simon lake ochinographer willam bb poler traveler sar earnist shaleton,for many then this book has been a source of fascination surely one of the most influential novels ever written an inspiration for such scientists and discoverers as engineer simon lake oceanographer william beebe polar traveler sir ernest shackleton
7,tabby had tended them in their childhood they and none others should tend her in her infirmity and age,tabby had tended them in their childhood they and none other should tend her in her infirmity and age
8,mistress fitzouth had been carried off bo the sheriff's daughter and her maids as soon as they had entered the house so that robin alone wat the care of mont fichet,mistress fitzooth had been carried off by the sheriff's daughter and her maids as soon as they had entered the house so that robin alone had the care of montfichet
9,said captain donnethorne seating himself where he could see along the short passage to the open derry door,said captain donnithorne seating himself where he could see along the short passage to the open dairy door


In [33]:
model.to("cuda")

with torch.no_grad():
  logits = model(torch.tensor(libri["test"][:1]["input_values"], device="cuda")).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist()))

/home/ujan/anaconda3/envs/asr/lib/python3.10/site-packages/torch/utils/checkpoint.py:25: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


'[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] c [PAD] [PAD] [PAD] [PAD] [PAD] o o o [PAD] n n [PAD] [PAD] [PAD] c c [PAD] [PAD] [PAD] [PAD] o o o r r r r d d | | r r [PAD] e [PAD] [PAD] [PAD] t t [PAD] [PAD] [PAD] [PAD] [PAD] u u r n n e d | | t o o | | | i [PAD] t s s s | | | p p l l [PAD] [PAD] [PAD] a a [PAD] c c e e [PAD] | | [PAD] [PAD] a [PAD] [PAD] [PAD] m m i i i d d d s s t t | | | t t h e e | | t t [PAD] [PAD] [PAD] [PAD] [PAD] e e n n n n [PAD] t [PAD] s s [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [34]:
torch.save(model.state_dict(), '/home/ujan/Notebooks/models/wav2vec2_asr_libri_100')